In [1]:
from transformers import LlamaTokenizerFast, BertTokenizerFast, AutoTokenizer
from tqdm import tqdm
from datasets import load_dataset

tokenizer_llama = LlamaTokenizerFast.from_pretrained("hf-internal-testing/llama-tokenizer")
# tokenizer_bloom = AutoTokenizer.from_pretrained("bigscience/bloom-560m")
# tokenizer_Bert = BertTokenizerFast.from_pretrained("google-bert/bert-base-uncased")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.54k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message.


In [3]:

def calculate_corpus_fertility(dataset_name, language_code, tokenizer, split='dev', max_docs=10000):
    """
    Calculate fertility for a dataset and tokenizer.

    Args:
        dataset_name (str): The dataset to load.
        language_code (str): The specific language code for the dataset.
        tokenizer_name (str): The tokenizer to use.
        split (str): The split of the dataset to evaluate (default: "train").
        max_docs (int): Maximum number of documents to evaluate (default: 10,000).

    Returns:
        float: The fertility score for the tokenizer on the dataset.
    """
    # Load dataset and tokenizer
    dataset = load_dataset(dataset_name, language_code, split=split)

    total_tokens = 0
    total_words = 0

    # Process up to max_docs
    for i, example in enumerate(tqdm(dataset, desc=f"Calculating fertility for {language_code}")):
        if i >= max_docs:
            break

        # Combine all text fields
        question = example["question"]
        choices = " ".join(eval(example["choices"]))  # Combine choices into a string
        answer = example["answer"]
        combined_text = f"{question} {choices} {answer}"

        # Tokenize and count tokens
        tokenized = tokenizer.tokenize(combined_text)
        total_tokens += len(tokenized)

        # Count words using whitespace splitting
        total_words += len(combined_text.split())

    # Compute fertility
    if total_words == 0:
        raise ValueError("Total words in the dataset is zero. Cannot compute fertility.")

    fertility = total_tokens / total_words
    return fertility

# Example usage
dataset_name = "masakhane/afrimmlu"
language_code = "eng"  # Example: Swahili
split = "dev"  # Use "train" or any available split
max_docs = 10000  # Use a held-out set of 10,000 documents

fertility_score = calculate_corpus_fertility(dataset_name, language_code, tokenizer_llama, split=split, max_docs=max_docs)
print(fertility_score)

README.md:   0%|          | 0.00/5.15k [00:00<?, ?B/s]

data/eng/val.tsv:   0%|          | 0.00/23.6k [00:00<?, ?B/s]

data/eng/dev.tsv:   0%|          | 0.00/7.99k [00:00<?, ?B/s]

data/eng/test.tsv:   0%|          | 0.00/136k [00:00<?, ?B/s]

Generating validation split:   0%|          | 0/83 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/25 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/500 [00:00<?, ? examples/s]

Calculating fertility for eng: 100%|██████████| 25/25 [00:00<00:00, 438.16it/s]

1.4142628205128205


In [6]:
languages = {
    "amh": "Amharic",
    "eng": "English",
    "ewe": "Ewe",
    "fra": "French",
    "hau": "Hausa",
    "ibo": "Igbo",
    "kin": "Kinyarwanda",
    "lin": "Lingala",
    "lug": "Luganda",
    "orm": "Oromo",
    "sna": "Shona",
    "sot": "Southern Sotho",
    "swa": "Swahili",
    "twi": "Twi",
    "wol": "Wolof",
    "xho": "Xhosa",
    "yor": "Yoruba",
    "zul": "Zulu",
}
results_Bloom_Tokenizer = {}
for lang_code, lang_name in languages.items():
    try:
        print(f"Processing language: {lang_name} ({lang_code})")
        fertility_score = calculate_corpus_fertility(
            dataset_name, lang_code, tokenizer_llama, split='test', max_docs=max_docs
        )
        results_Bloom_Tokenizer[lang_code] = fertility_score
        print(f"Fertility Score for {lang_name}: {fertility_score}")
    except Exception as e:
        print(f"Failed for {lang_name} ({lang_code}): {e}")

# Print all results
print("\nFertility Scores by Language:")
for lang_code, score in results_Bloom_Tokenizer.items():
    print(f"{languages[lang_code]} ({lang_code}): {score}")

Processing language: Amharic (amh)


data/amh/val.tsv:   0%|          | 0.00/35.7k [00:00<?, ?B/s]

data/amh/dev.tsv:   0%|          | 0.00/11.8k [00:00<?, ?B/s]

data/amh/test.tsv:   0%|          | 0.00/211k [00:00<?, ?B/s]

Generating validation split:   0%|          | 0/83 [00:00<?, ? examples/s]

...

Generating validation split:   0%|          | 0/83 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/25 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/500 [00:00<?, ? examples/s]

Calculating fertility for zul: 100%|██████████| 500/500 [00:00<00:00, 1179.66it/s]

Fertility Score for Zulu: 3.8692230743581195

Fertility Scores by Language:
Amharic (amh): 11.52529601722282
English (eng): 1.475627916211655
Ewe (ewe): 2.970092121314096
French (fra): 1.8337786826144125
Hausa (hau): 2.550557802569003
Igbo (ibo): 2.965458006406983
Kinyarwanda (kin): 3.2254943652987453
Lingala (lin): 2.0521419207510467
Luganda (lug): 3.521403576051004
Oromo (orm): 3.381484265922028
Shona (sna): 3.419693445458341
Southern Sotho (sot): 2.2515918230563003
Swahili (swa): 2.5851301823028225
Twi (twi): 2.542355665705115
Wolof (wol): 2.289836977847537
Xhosa (xho): 3.9811791689152725
Yoruba (yor): 3.638583532478818
Zulu (zul): 3.8692230743581195
